# Step I - Import Libraries and get the list of all equity derivatives

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as pdr
import talib as ta

C:\Users\akshay\anaconda3\envs\algo\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
stocks = pd.read_csv('NSE_derivatives_stocks.csv',encoding= 'unicode_escape')

In [3]:
ticker = stocks['ticker']

In [4]:
ticker = [item+'.NS' for item in ticker]

# Step II - Fetch data from Yahoo Finance API

In [5]:
start = dt.datetime.today() - dt.timedelta(360)
end = dt.datetime.today()

In [6]:
data = pdr.get_data_yahoo(ticker,start,end)

In [7]:
data_close = data['Adj Close']
price = data_close.copy()

In [8]:
price_pct = price.pct_change(1)

# Step III - Apply the scans

# 1. Price Change > 3%

In [9]:
scanned_stocks = []

In [10]:
for i in ticker:
    if price_pct[i][-1] > 0.03:
        scanned_stocks.append(i)

# 2. Price > 200 SMA and RSI(2) > 50

In [11]:
final_stocks = []

In [12]:
for i in scanned_stocks:
    if (ta.SMA(price[i],200)[-1] < price[i][-1]) & (ta.RSI(price[i],2)[-1] > 50):
        final_stocks.append(i)

In [13]:
cp_stocks = price[final_stocks].iloc[-1].round(1)

# Step IV - Find the Entry, Stop loss and Target price

In [14]:
entry_price = (cp_stocks*1.01).round(1)
entry_price = pd.DataFrame(entry_price)
sl_price = (entry_price* 1.03).round(1)
tgt_price = (entry_price* 0.94).round(1)

In [15]:
order_df = pd.concat([entry_price,sl_price,tgt_price],axis =1,ignore_index = True)

In [16]:
order_df.rename(columns = {0:'Entry Price', 1:'Stop Loss', 2:'Target'},inplace = True)

In [17]:
order_df

,Entry Price,Stop Loss,Target
Symbols,,,
BALKRISIND.NS,936.1,964.2,879.9
IGL.NS,501.0,516.0,470.9


In [18]:
trade_date = str(end + dt.timedelta(1))

In [19]:
order_df.to_csv('trade-data/.csv')